# In-class notebook about sorting

In this notebook, we will talk a little about common speed-up strategies in sorting and searching.

**Why we care:** sorting comes up often in prorgamming and in astrophyscs-- for example if you have a catalogue of galaxies and you want to identify the galaxy clusters you must organize the caltalouge such that nearboring galaxies are grouped together.

This notebook is intended to support Chapter 1-2 of the textbook.   See the [Data Set Examples](https://www.astroml.org/examples/datasets/index.html) for AstroML to explore and work the excercises.

Material is taken from the following scripts (from astroML):
* https://github.com/astroML/astroML_figures/blob/main/book_figures/chapter2/fig_search_scaling.py
* https://github.com/astroML/astroML_figures/blob/main/book_figures/chapter2/fig_sort_scaling.py
* https://github.com/astroML/astroML_figures/blob/main/book_figures/chapter2/fig_kdtree_example.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Sorting: numpy quick-sort and python list sort

Here we run a simple sort and use the "%time" magic command to bench mark its execution 

In [ ]:
# generate a random, uniformly distributed array
np.random.seed(0)
X = np.random.rand(10000000)
print(X)

In [ ]:
%time X.sort()  ## this sorts in place-- so X after this is run X is now ordered

In [ ]:
# note that a pre-sorted array sorts faster!
%time X.sort()

In [ ]:
print(X)  ## see that X is ordered... 

### Compare two types of sorting

In [ ]:
from time import time


###########################################################################
## benchmark two sorting routines (inhareted from the array type in python)

# time quick-sort of a numpy array
N_npy = 10 ** np.linspace(5, 7, 10)  ## 10 logrytmic bins from 10^5 to 10^7
time_npy = np.zeros_like(N_npy)


#### the key point is that this is a numpy array (see arrow)
for i in range(len(N_npy)):
    x = np.random.random(int(N_npy[i]))   ### <<<<---------
    t0 = time()
    x.sort()
    t1 = time()
    time_npy[i] = t1 - t0

# time built-in sort of python list
N_list = N_npy.copy()
time_list = np.zeros_like(N_list)

#### the key point is that this is a python list (see arrow)
for i in range(len(N_list)):
    x = list(np.random.random(int(N_list[i])))  ### <<<<---------
    t0 = time()
    x.sort()
    t1 = time()
    time_list[i] = t1 - t0


###########################################################################
## plot the results 

fig = plt.figure(figsize=(5, 3.75))
fig.subplots_adjust(bottom=0.15)
ax = plt.axes(xscale='log', yscale='log')
ax.grid()

# plot the observed times
ax.plot(N_list, time_list, 'sk', color='gray', ms=5, label='list sort')
ax.plot(N_npy, time_npy, 'ok', color='gray', ms=5, label='NumPy sort')

# plot the expected scalings
scale = np.linspace(N_npy[0] / 2, N_npy[-1] * 2, 100)
scaling_N = scale * time_npy[0] / N_npy[0]
scaling_NlogN = (scale * np.log2(scale) 
                 * time_npy[0] / N_npy[0] / np.log2(N_npy[0]))

ax.plot(scale, scaling_NlogN, '--k', label=r'$\mathcal{O}[N \log N]$')
ax.plot(scale, scaling_N, ':k', label=r'$\mathcal{O}[N]$')

scaling_N = scale * time_list[0] / N_list[0]
scaling_NlogN = (scale * np.log2(scale) * time_list[0]
                 / N_list[0] / np.log2(N_list[0]))

ax.plot(scale, scaling_NlogN, '--k')
ax.plot(scale, scaling_N, ':k')

# Create titles and labels
ax.set_title("Scaling of Sort Algorithms")
ax.set_xlabel('Length of Array')
ax.set_ylabel('Relative sort time')
plt.legend(loc='upper left')

ax.set_xlim(scale[0], scale[-1])


Which data strucure makes for more efficiect sorting?  

### Searching: linear search and binary search in an ordered list 

often we want to find an element (or elements) in an array that satifsy some condition.  If we pre-sort the array we can take advantage of this strucure to singificnatly boost the speed.  

here we compare two fucntions: np.where-- a linear search, and np.searhsorted-- a binary search on ordeered data.

In [ ]:
# Compute the execution times as a function of array size
Nsamples = 10 ** np.linspace(6.0, 7.8, 17)  ## 17 logrythmic bins from 10^6 to 10^7.8
time_linear = np.zeros_like(Nsamples)  ## time for linear search
time_binary = np.zeros_like(Nsamples)  ## time for binary search

for i in range(len(Nsamples)):
    # create a sorted array
    x = np.arange(Nsamples[i], dtype=int)

    # Linear search: choose a single item in the array
    item = int(0.4 * Nsamples[i])

    t0 = time()
    j = np.where(x == item)   ## this is one of my favorite functions-- but it is slow comapred to binary (see below)
    t1 = time()

    time_linear[i] = t1 - t0

    # Binary search: this is much faster, so choose 1000 items to search for
    items = np.linspace(0, Nsamples[i], 1000).astype(int)

    t0 = time()
    j = np.searchsorted(x, items)
    t1 = time()

    time_binary[i] = (t1 - t0)



########################################################
### make the plots

fig = plt.figure(figsize=(5, 3.75))
fig.subplots_adjust(bottom=0.15)
ax = plt.axes(xscale='log', yscale='log')
ax.grid()

# plot the observed times
ax.plot(Nsamples, time_linear, 'o', color='gray', markersize=5,
        label=r'linear search $(\mathcal{O}[N])$')
ax.plot(Nsamples, time_binary, 's', color='gray', markersize=5,
        label=r'efficient search $(\mathcal{O}[\log N])$')

# plot the expected scaling
scale = 10 ** np.linspace(5, 8, 100)
scaling_N = scale * time_linear[7] / Nsamples[7]
scaling_logN = np.log(scale) * time_binary[7] / np.log(Nsamples[7])
ax.plot(scale, scaling_N, '--k')
ax.plot(scale, scaling_logN, '--k')

ax.set_xlim(9E5, 1E8)

# add text and labels
ax.set_title("Scaling of Search Algorithms")
ax.set_xlabel('Length of Array')
ax.set_ylabel('Relative search time')
ax.legend(loc='upper left')
plt.show()

note the massive improvement offered by binary search

### Searching: nearest-neighbor search

We want to find the nearest point to each point in an array.  we will compare two versions of the code.  One where we use loops and a second where we vectorize.  The lession is always vectroize.

In [ ]:
# we look at two implemntaitons of code that calculates for each 
# entry in an array computes the distance to all other points and finds the minimum
# it then returns the index for the closest neighbor


## first implementation using loops
def easy_nn(X):
    N, D = X.shape
    neighbors = np.zeros(N, dtype=int)
    for i in range(N):
        j_closest = i
        d_closest = np.inf # initialize closest distance to infinity
        for j in range(N):
            if i==j:
                continue
            d = np.sqrt(np.sum((X[i] - X[j])**2))
            if d < d_closest:
                j_closest = j 
                d_closest = d
        neighbors[i] = j_closest
    return neighbors

## second implmentaiton using vector operaitons
def vectorized_nn(X):
    XXT = np.dot(X, X.T)
    Xii = XXT.diagonal()
    D = np.sqrt(Xii - 2* XXT + Xii[:, np.newaxis])
    return np.argsort(D, axis=1)[:,1] # first element is 0

In [ ]:
X = np.random.random((10,3))
print(X)

In [ ]:
easy_nn(X)

In [ ]:
vectorized_nn(X)

In [ ]:
%timeit easy_nn(X)

In [ ]:
%timeit vectorized_nn(X)

In [ ]:
X = np.random.random((1000,3))

%timeit easy_nn(X)

%timeit vectorized_nn(X)

by what ratio is the vectorized version faster than the implementaiton based on loops?

### Finally, let's look at the idea of "trees"

Assume we have a similar problem where we want to look for nearest neighbors in an array X.  

We will use the [K-dimentional tree (KDtree](https://en.wikipedia.org/wiki/K-d_tree) algorythem.  It is a method for organizing the data for efficienct search.

this is a divide and conquor apprach.

In [ ]:
X = np.random.random((10,3))
print(X)

In [ ]:
from scipy.spatial import cKDTree

kdt = cKDTree(X) # build KD tree here

In [ ]:
vectorized_nn(X)

In [ ]:
kdt.query(X, k=2)[1][:,1] # first array is distance, second is index
                          # show first and second closest neighbor, 
                          # where first will just be itself

In [ ]:
kdt.query(X, k=2)

In [ ]:
%timeit kdt.query(X, k=2)

### Lets look at what KD trees are actually doing with a 2D example

In [ ]:
# Create a set of structured random points in two dimensions
np.random.seed(0)

X = np.random.random((30, 2)) * 2 - 1
X[:, 1] *= 0.1
X[:, 1] += X[:, 0] ** 2

plt.figure(figsize=(3,3))
plt.scatter(X[:,0], X[:,1])
plt.xlim(-1.1,1.1)
plt.ylim(-0.1,1.1)

In [ ]:
# Create a KDTree class which will recursively subdivide the
# space into rectangular regions.  Note that this is just an example
# and shouldn't be used for real computation; instead use the optimized
# code in scipy.spatial.cKDTree or sklearn.neighbors.BallTree

class KDTree:
    """Simple KD tree class"""

    # class initialization function
    def __init__(self, data, mins, maxs):
        self.data = np.asarray(data)

        # data should be two-dimensional
        assert self.data.shape[1] == 2

        if mins is None:
            mins = data.min(0)
        if maxs is None:
            maxs = data.max(0)

        self.mins = np.asarray(mins)
        self.maxs = np.asarray(maxs)
        self.sizes = self.maxs - self.mins

        self.child1 = None
        self.child2 = None

        if len(data) > 1:
            # sort on the dimension with the largest spread (this alternates in this example)
            largest_dim = np.argmax(self.sizes)
            i_sort = np.argsort(self.data[:, largest_dim])
            self.data[:] = self.data[i_sort, :]

            # find split point, each time splitting half the objects
            N = self.data.shape[0]
            half_N = int(N / 2)
            split_point = 0.5 * (self.data[half_N, largest_dim]
                                 + self.data[half_N - 1, largest_dim])

            # create subnodes (form a line in the plane)
            mins1 = self.mins.copy()
            mins1[largest_dim] = split_point
            maxs2 = self.maxs.copy()
            maxs2[largest_dim] = split_point

            # Recursively build a KD-tree on each sub-node
            self.child1 = KDTree(self.data[half_N:], mins1, self.maxs)
            self.child2 = KDTree(self.data[:half_N], self.mins, maxs2)

    def draw_rectangle(self, ax, depth=None):
        """Recursively plot a visualization of the KD tree region"""
        if depth == 0:
            rect = plt.Rectangle(self.mins, *self.sizes, ec='k', fc='none')
            ax.add_patch(rect)

        if self.child1 is not None:
            if depth is None:
                self.child1.draw_rectangle(ax)
                self.child2.draw_rectangle(ax)
            elif depth > 0:
                self.child1.draw_rectangle(ax, depth - 1)
                self.child2.draw_rectangle(ax, depth - 1)


In [ ]:
#------------------------------------------------------------
# Use our KD Tree class to recursively divide the space
KDT = KDTree(X, [-1.1, -0.1], [1.1, 1.1])

#------------------------------------------------------------
# Plot four different levels of the KD tree
fig = plt.figure(figsize=(10, 5))
fig.subplots_adjust(wspace=0.1, hspace=0.15,
                    left=0.1, right=0.9,
                    bottom=0.05, top=0.9)

for level in range(1, 7):
    ax = fig.add_subplot(2, 3, level, xticks=[], yticks=[])
    ax.scatter(X[:, 0], X[:, 1], s=9)
    KDT.draw_rectangle(ax, depth=level - 1)

    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-0.15, 1.15)
    ax.set_title('level %i' % level)

# suptitle() adds a title to the entire figure
fig.suptitle('$k$d-tree Example')